# This is for Level 2 ensemble over all Level 1 model outputs

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

In [3]:
# A list of predictions that we ensemble
lgbm_path='../data/'

lgbm0=pd.read_csv(lgbm_path+'train_lgbm_phase5_new.csv')
lgbm1=pd.read_csv(lgbm_path+'train_lgbm_phase5_model1.csv')
lgbm2=pd.read_csv(lgbm_path+'train_lgbm_phase5_model2.csv')

In [4]:
dnn_path='./jl4305/backup/'

dnn0=pd.read_csv(dnn_path+'output_float_phase5_epoch_10.csv')
dnn1=pd.read_csv(dnn_path+'output_float_phase5_epoch_20.csv')

In [5]:
dnn0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 2 columns):
id        int64
target    float64
dtypes: float64(1), int64(1)
memory usage: 112.6 MB


In [6]:
# Concatenate columns
train=pd.DataFrame(index=lgbm0.index)

train['lgbm0']=lgbm0.target
train['lgbm1']=lgbm1.target
train['lgbm2']=lgbm2.target

train['dnn0']=dnn0.target
train['dnn1']=dnn1.target

In [7]:
# Read test columns
lgbm_test0=pd.read_csv(lgbm_path+'submission_lgbm_phase5_new.csv')
lgbm_test1=pd.read_csv(lgbm_path+'train_lgbm_phase5_model1.csv')
lgbm_test2=pd.read_csv(lgbm_path+'train_lgbm_phase5_model2.csv')

dnn_test0=pd.read_csv(dnn_path+'train_phase5_epoch10.csv')
dnn_test1=pd.read_csv(dnn_path+'train_phase5_epoch20.csv')

In [8]:
test=pd.DataFrame(index=lgbm_test0.index)

test['lgbm0']=lgbm_test0.target
test['lgbm1']=lgbm_test1.target
test['lgbm2']=lgbm_test2.target

test['dnn0']=dnn_test0.target
test['dnn1']=dnn_test1.target

In [9]:
# Load example answer
data_path='../data/'
example=pd.read_csv(data_path + 'train_phase5.csv', index_col=0)

/home/jiacheliu3/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
y=example['target']

In [11]:
y.describe()

count    7.377418e+06
mean     5.035171e-01
std      4.999877e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: target, dtype: float64

In [30]:
lr=LinearRegression(normalize=True)
lr.fit(train,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [31]:
pred=lr.predict(test)

In [35]:
# Normalize to 0-1
pred=(pred-np.min(pred))/(np.max(pred)-np.min(pred))

In [37]:
pred_df=pd.DataFrame()
pred_df['id']=test.index
pred_df['target']=pred
pred_df.to_csv(data_path+'ensemble_phase5_lr.csv', index=False)

# Print the coefficients used in Level 2 model to see the importance of Level 1 models

In [33]:
lr.coef_

array([ 1.61349017, -0.84253125,  0.29199245, -1.52670813,  1.48192019])